<a href="https://www.kaggle.com/code/hari141v/semantic-search-vector-evaluation-elasticsearch?scriptVersionId=188108714" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### About
### Information retrieval using elasticsearch with vector. Calculate recall, mrr from search results
### The code is copied from [LLM_ZOOMCAMP](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluate-vector.ipynb)

### Import necessary libraries and packages

In [1]:
pip install elasticsearch -qq 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence_transformers==2.7.0 -qq

Note: you may need to restart the kernel to use updated packages.


In [3]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.14.0-linux-x86_64.tar.gz
!chown -R root:root elasticsearch-8.14.0
!adduser --disabled-password --gecos '' elasticsearch_user
!chown -R elasticsearch_user:elasticsearch_user elasticsearch-8.14.0

--2024-07-13 13:58:21--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588482330 (561M) [application/x-gzip]
Saving to: 'elasticsearch-8.14.0-linux-x86_64.tar.gz'

elasticsearch-8.14. 100%[===================>] 561.22M  14.1MB/s    in 33s     

2024-07-13 13:58:55 (17.0 MB/s) - 'elasticsearch-8.14.0-linux-x86_64.tar.gz' saved [588482330/588482330]

Adding user `elasticsearch_user' ...
Adding new group `elasticsearch_user' (1000) ...
Adding new user `elasticsearch_user' (1001) with group `elasticsearch_user' ...
Creating home directory `/home/elasticsearch_user' ...
Copying files from `/etc/skel' ...


In [55]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io
import os
import json
import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json
import elasticsearch
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from IPython.display import clear_output

### Elasticsearch setup

In [99]:
# Run elasticsearch
command = "sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(20)

In [100]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

root        1180      33  0 15:02 ?        00:00:00 /bin/sh -c sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
root        1181    1180  0 15:02 ?        00:00:00 sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
elastic+    1182    1181  8 15:02 ?        00:00:06 /kaggle/working/elasticsearch-8.14.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.14.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/kaggle/working/elasticsearch-8.14.0 -Des.path.conf=/kaggle/working/elasticsearch-8.14.0/config -Des.distribution.type=tar -cp /kaggle/working/elasticsearch-8.14.0/lib/*:/kaggle/working/elasticsearch-8.14.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -E xpack.security.enabled=false -E thread_pool.se

In [101]:
es_client = Elasticsearch('http://localhost:9200/', request_timeout=60) 

In [102]:
!curl localhost:9200

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{
  "name" : "d7691267410d",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "9IeJ6I8IQYOO26MWCv1bPQ",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### Get document file from coure repo

In [11]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json

--2024-07-13 14:00:22--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 699257 (683K) [text/plain]
Saving to: 'documents-with-ids.json'

documents-with-ids. 100%[===================>] 682.87K  3.75MB/s    in 0.2s    

2024-07-13 14:00:23 (3.75 MB/s) - 'documents-with-ids.json' saved [699257/699257]



### Load the document

In [12]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

### Get sentence transformer model

In [16]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Sample Encode
#### By default, the embedding vector is not normalized.
#### Use normalize_embeddings=True for cosine similarity search.
#### For dot-product similarity search, normalization is not necessary.

In [19]:
v = model.encode('I just discovered the course. Can I still join?',normalize_embeddings=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Embedding dimension

In [23]:
v.shape

(384,)

### Create index for elasticsearch

In [103]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Encode the question, text, and combined text+question and add to the document

In [26]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question, normalize_embeddings=True, show_progress_bar=False)
    doc['text_vector'] = model.encode(text, normalize_embeddings=True, show_progress_bar=False)
    doc['question_text_vector'] = model.encode(qt, normalize_embeddings=True, show_progress_bar=False)

  0%|          | 0/948 [00:00<?, ?it/s]

### Document after adding vectors

In [110]:
for k in documents[0].keys():
    if k in ['question_vector', 'text_vector', 'question_text_vector']:
        print(k,':',len(documents[0][k]))
    else:
        print(k,':',documents[0][k])

text : The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.
section : General course-related questions
question : Course - When will the course start?
course : data-engineering-zoomcamp
id : c02e79ef
question_vector : 384
text_vector : 384
question_text_vector : 384


### Add index to elasticsearch

In [111]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Sample search

In [43]:
query = 'I just discovered the course. Can I still join it?'

In [44]:
v_q = model.encode(query, normalize_embeddings=True, show_progress_bar=False)

In [85]:
q={
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000, 
}
es_client.search(index=index_name, knn=q, 
                 source=["text", "section", "question", "course"])["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'yb-MrJAByqc4kY5inIfS',
  '_score': 0.7878411,
  '_source': {'text': 'The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well).',
   'section': 'General course-related questions',
   'question': 'When does the next iteration start?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'yL-MrJAByqc4kY5inIfB',
  '_score': 0.7668669,
  '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for 

### KNN vector search

In [68]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

### Get ground truth data(LLM generated question data) from course repo

In [48]:
df_ground_truth_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/ground-truth-data.csv'
df_ground_truth=pd.read_csv(df_ground_truth_url)
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


### Convert each row to dictionary

In [49]:
ground_truth = df_ground_truth.to_dict(orient='records')

### Hit-rate metric

In [51]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

### MRR metric

In [52]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [70]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

### Function to embed question

In [ ]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question, normalize_embeddings=True, show_progress_bar=False)

    return elastic_search_knn('question_vector', v_q, course)

### Evaluation result for question vector

In [71]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

ES text only: 0.7395720769397017, 0.6032418413658963

### Function to embed text

In [120]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question,normalize_embeddings=True, show_progress_bar=False)

    return elastic_search_knn('text_vector', v_q, course)

### Evaluation result for text vector

In [121]:
evaluate(ground_truth, text_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7810676464231684, 'mrr': 1.0101109430156314}

### Function to embed combined text and question

In [91]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question,normalize_embeddings=True, show_progress_bar=False)

    return elastic_search_knn('question_text_vector', v_q, course)



### Evaluation result for combined text and question

In [92]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

### Custom scoring based on the cosine similarity between text, question, text+question vectors

In [93]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

### Search based on combined vector similarity

In [112]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question,normalize_embeddings=True, show_progress_bar=False)

    return elastic_search_knn_combined(v_q, course)



### Evaluation result for combined vector similarity

In [113]:
evaluate(ground_truth, vector_combined_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.869894099848714, 'mrr': 1.1683956487284752}